# 03 — Normalize Vicon QdM by Shoulder Width

This notebook merges:
- `vicon_QDM_wrists_head_mm.xlsx` (movement quantities in mm)
- `vicon_shoulder_width_by_csv.xlsx` (median shoulder width in mm)

Then it computes normalized movement indices (unitless):
- `QDM_WRISTS_norm = QDM_WRISTS_mm / shoulder_width_median_mm`
- `QDM_HEAD_norm   = QDM_HEAD_mm / shoulder_width_median_mm`

Outputs:
- `results/<MODE>/vicon_QDM_wrists_head_normByShoulders.xlsx`

## 1) Project paths (MODE-aware)

We support two modes:
- `test`: small dataset tracked in GitHub
- `raw`: full dataset (typically ignored by git)

All outputs are written to: `results/<MODE>/`

In [7]:
from pathlib import Path
import pandas as pd


# Mode (test vs raw) + paths

MODE = "raw"   # change to "raw" for full dataset

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
RESULTS_DIR = PROJECT_ROOT / "results" / MODE
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Mode:", MODE)
print("Project root:", PROJECT_ROOT)
print("Results dir:", RESULTS_DIR.resolve())

Mode: raw
Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw


## 2) Load intermediate Excel outputs

Inputs (generated by previous notebooks):
- `vicon_QDM_wrists_head_mm.xlsx`
- `vicon_shoulder_width_by_csv.xlsx`

In [8]:
qdm_path = RESULTS_DIR / "vicon_QDM_wrists_head_mm.xlsx"
sw_path  = RESULTS_DIR / "vicon_shoulder_width_by_csv.xlsx"

qdm = pd.read_excel(qdm_path)
sw  = pd.read_excel(sw_path)

print("QDM rows:", len(qdm), "| columns:", qdm.shape[1])
print("SW rows:", len(sw),  "| columns:", sw.shape[1])
qdm.head()

QDM rows: 36 | columns: 22
SW rows: 36 | columns: 8


,csv,file,P1_poignet_D_QDM_mm,P1_poignet_D_nsteps,P1_poignet_G_QDM_mm,P1_poignet_G_nsteps,P1_QDM_WRISTS_mm,P1_Tempe_D_QDM_mm,P1_Tempe_D_nsteps,P1_Tempe_G_QDM_mm,...,P2_2poignet_D_QDM_mm,P2_2poignet_D_nsteps,P2_2poignet_G_QDM_mm,P2_2poignet_G_nsteps,P2_QDM_WRISTS_mm,P2_2Tempe_D_QDM_mm,P2_2Tempe_D_nsteps,P2_2Temps_G_QDM_mm,P2_2Temps_G_nsteps,P2_QDM_HEAD_mm
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,11257.605164,18011,5086.439843,18011,16344.045006,15748.775325,18011,18437.193954,...,6990.478950,18011,4791.810419,18011,11782.289369,10070.014078,18011,10379.521199,18011,20449.535277
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD02.csv,15422.894012,17999,16779.292373,17999,32202.186385,9725.644869,17999,10354.212811,...,16420.189086,17999,17622.889557,17999,34043.078643,9851.813433,17999,10217.118782,17999,20068.932215
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD03.csv,34870.992267,17999,27170.831678,17999,62041.823945,14105.980148,17999,14473.346651,...,19508.827550,17999,6708.991777,17999,26217.819326,8916.973687,17999,9655.490833,17999,18572.464520
3,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD04.csv,2661.923274,17999,3833.346976,17999,6495.270250,4847.753225,17999,4768.206439,...,7680.839058,17999,11250.402281,17999,18931.241339,7442.981553,17999,7080.281923,17999,14523.263476
4,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD05.csv,6273.538429,17999,6588.362586,17999,12861.901015,4558.721874,17999,4829.197593,...,2684.099411,17999,5080.767751,17999,7764.867162,5805.341913,17999,5511.189012,17999,11316.530925


## 3) Build a shared key (`video_id`)

We create `video_id` to merge the two tables.
- QDM uses `file` (e.g., `SEATEDD01.csv`)
- Shoulder-width may store either `csv` (full path) or `file`

In [9]:
# QDM: column 'file' like "SEATEDD01.csv"
qdm["video_id"] = qdm["file"].astype(str).str.replace(".csv", "", regex=False)

# Shoulder-width: column 'file' also exists in our pipeline, but we keep your robust approach:
# if "csv" contains the full path, use stem (filename without extension)
if "csv" in sw.columns:
    sw["video_id"] = sw["csv"].astype(str).apply(lambda x: Path(x).stem)
else:
    # fallback if sw already has "file" like "SEATEDD01.csv"
    sw["video_id"] = sw["file"].astype(str).str.replace(".csv", "", regex=False)

qdm[["file", "video_id"]].head()

,file,video_id
0,SEATEDD01.csv,SEATEDD01
1,SEATEDD02.csv,SEATEDD02
2,SEATEDD03.csv,SEATEDD03
3,SEATEDD04.csv,SEATEDD04
4,SEATEDD05.csv,SEATEDD05


## 4) Merge shoulder width into QDM and check missing values

In [10]:
out = qdm.merge(
    sw[[
        "video_id",
        "P1_shoulder_width_median_mm",
        "P2_shoulder_width_median_mm"
    ]],
    on="video_id",
    how="left"
)

missing = out["P1_shoulder_width_median_mm"].isna().sum()
print(f"Rows without shoulder width: {missing}/{len(out)}")

if missing > 0:
    display(out.loc[out["P1_shoulder_width_median_mm"].isna(), ["video_id"]].head(20))

Rows without shoulder width: 0/36


## 5) Compute normalized movement indices

Normalization:
- Wrist QdM / shoulder width
- Head QdM / shoulder width

Outputs are unitless (a.u.).

In [11]:
# Wrists
out["P1_QDM_WRISTS_norm"] = out["P1_QDM_WRISTS_mm"] / out["P1_shoulder_width_median_mm"]
out["P2_QDM_WRISTS_norm"] = out["P2_QDM_WRISTS_mm"] / out["P2_shoulder_width_median_mm"]

# Head
out["P1_QDM_HEAD_norm"] = out["P1_QDM_HEAD_mm"] / out["P1_shoulder_width_median_mm"]
out["P2_QDM_HEAD_norm"] = out["P2_QDM_HEAD_mm"] / out["P2_shoulder_width_median_mm"]

out[[
    "video_id",
    "P1_QDM_WRISTS_mm", "P1_shoulder_width_median_mm", "P1_QDM_WRISTS_norm",
    "P2_QDM_WRISTS_mm", "P2_shoulder_width_median_mm", "P2_QDM_WRISTS_norm"
]].head()

,video_id,P1_QDM_WRISTS_mm,P1_shoulder_width_median_mm,P1_QDM_WRISTS_norm,P2_QDM_WRISTS_mm,P2_shoulder_width_median_mm,P2_QDM_WRISTS_norm
0,SEATEDD01,16344.045006,281.600411,58.039848,11782.289369,305.506314,38.566435
1,SEATEDD02,32202.186385,286.055610,112.573169,34043.078643,305.121583,111.572175
2,SEATEDD03,62041.823945,289.523437,214.289470,26217.819326,262.377859,99.923901
3,SEATEDD04,6495.270250,284.671404,22.816729,18931.241339,348.361032,54.343740
4,SEATEDD05,12861.901015,263.954779,48.727669,7764.867162,361.325059,21.489977


## 6) Export

In [12]:
out_path = RESULTS_DIR / "vicon_QDM_wrists_head_normByShoulders.xlsx"
out.to_excel(out_path, index=False)

print("✅ Saved:", out_path.resolve())

✅ Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw/vicon_QDM_wrists_head_normByShoulders.xlsx
